# Feature Extraction

In [1]:
from keras.applications import VGG16
#VGG16 : Convolution layers 13 / fully connected layers 3
#VGG19 : Convolution layers 16 / fully connected layers 3

In [2]:
conv_base = VGG16(weights = 'imagenet' , #초기화할 가중치 체크포인트 지정
                 include_top = False, #최상위 완전 연결 분규리를 포함할지 안 할지(default 클래스 1000에 대응되는 분류기 포함)
                 input_shape = (150,150,3)) #네트워크에 주입할 이밎 텐서의 크기. 만약 include_top = True이면, 완전 연결 층이 추가되기 때문에 원본 모델과 동일해야 함

58892288/58889256 [==============================] - 28s 0us/step


In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

## not using Data Augmentation
- 새로운 데이터셋에서 CNN layer를 실행하고, 출력을 numpy array로 디스크에 저장.
- 독립된 완전 연결 분류기에 입력으로 사용.
- 모든 이미지에 CNN층을 한 번만 실행하면 되기 때문에 빠르고 비용이 적게 듬

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

## using Data Augmentation
- conv_base 위에 Dense 층을 쌓아 확장
- 입력 데이터에서 end-to-end 모델을 실행
- 모델에 노출된 모든 입력 이미지가 매번 CNN laeyr를 통과하기 때문에 데이터 증식 자으
- 비용이 많이 듬